In [2]:
import pandas as pd
import json

In [15]:
raw_data_path = r"data\test.json"
output_csv_path = r"data\flights.csv"

In [53]:
with open(raw_data_path) as f:
    data = json.load(f)["items"]

df = pd.DataFrame(data)
df["stops_num"] = df.stops.str["stops"]
max_stops = df["stops_num"].max()

df2 = pd.DataFrame(df.stops.str["stops_data"].tolist())

for i in range(max_stops):
    df[f"stop_{i+1}_duration"] = df2[i].str["duration"]
    df[f"stop_{i+1}_place"] = df2[i].str["place"]

print(df.stops.str["stops_data"])
df = df.drop('stops', axis=1)
df.sort_values("stops_num")

0     [{'stop_num': 0, 'duration': '1 hr 15 min', 'p...
1     [{'stop_num': 0, 'duration': '2 hr 5 min', 'pl...
2     [{'stop_num': 0, 'duration': '1 hr 55 min', 'p...
3     [{'stop_num': 0, 'duration': '23 hr 35 min', '...
4     [{'stop_num': 0, 'duration': '1 hr 50 min', 'p...
                            ...                        
63    [{'stop_num': 0, 'duration': '6 hr', 'place': ...
64    [{'stop_num': 0, 'duration': '2 hr 5 min', 'pl...
65    [{'stop_num': 0, 'duration': '55 min', 'place'...
66    [{'stop_num': 0, 'duration': '1 hr 55 min', 'p...
67    [{'stop_num': 0, 'duration': '45 min', 'place'...
Name: stops, Length: 68, dtype: object


,date_readed,departure_date,arrival_date,departure_time,arrival_time,return_date,price,airline,duration,stops_num,stop_1_duration,stop_1_place,stop_2_duration,stop_2_place
54,2022.10.13,"Friday, February 3 2023","Friday, February 3 2023",6:45 AM,9:30 AM,2023.02.07,1158,Wizz Air,2 hr 45 min,0,None,None,None,None
47,2022.10.13,"Friday, February 3 2023","Friday, February 3 2023",9:55 AM,12:40 PM,2023.02.07,831,Ryanair,2 hr 45 min,0,None,None,None,None
0,2022.10.13,"Thursday, February 2 2023","Thursday, February 2 2023",7:30 AM,12:05 PM,2023.02.06,608,Lufthansa,4 hr 35 min,1,1 hr 15 min,Munich,None,None
35,2022.10.13,"Thursday, February 2 2023","Thursday, February 2 2023",2:30 PM,7:45 PM,2023.02.06,723,Austrian,5 hr 15 min,1,1 hr 55 min,Vienna,None,None
36,2022.10.13,"Thursday, February 2 2023","Friday, February 3 2023",4:00 PM,8:25 PM,2023.02.06,624,easyJet,28 hr 25 min,1,23 hr 35 min,Bristol,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,2022.10.13,"Thursday, February 2 2023","Thursday, February 2 2023",1:25 PM,5:35 PM,2023.02.07,686,Lufthansa,4 hr 10 min,1,50 min,Munich,None,None
10,2022.10.13,"Thursday, February 2 2023","Thursday, February 2 2023",6:00 AM,2:00 PM,2023.02.06,1040,KL,8 hr,2,1 hr 30 min,Amsterdam,1 hr 20 min,Paris
44,2022.10.13,"Thursday, February 2 2023","Thursday, February 2 2023",6:00 AM,2:00 PM,2023.02.06,1340,KL,8 hr,2,1 hr 30 min,Amsterdam,1 hr 20 min,Paris
21,2022.10.13,"Thursday, February 2 2023","Thursday, February 2 2023",6:00 AM,2:00 PM,2023.02.07,1040,KL,8 hr,2,1 hr 30 min,Amsterdam,1 hr 20 min,Paris


In [19]:
df.drop_duplicates().sort_values("price").head()

,date_readed,departure_date,arrival_date,departure_time,arrival_time,return_date,price,airline,duration,stops_num,stops_place,stops_duration
48,2022.10.13,"Friday, February 3 2023","Saturday, February 4 2023",8:55 PM,9:20 AM,2023.02.07,607,easyJet,12 hr 25 min,None,None,None
45,2022.10.13,"Friday, February 3 2023","Friday, February 3 2023",1:25 PM,5:35 PM,2023.02.07,608,Lufthansa,4 hr 10 min,None,None,None
55,2022.10.13,"Friday, February 3 2023","Friday, February 3 2023",1:25 PM,5:35 PM,2023.02.08,608,Lufthansa,4 hr 10 min,None,None,None
22,2022.10.13,"Thursday, February 2 2023","Thursday, February 2 2023",7:30 AM,12:05 PM,2023.02.08,608,Lufthansa,4 hr 35 min,None,None,None
11,2022.10.13,"Thursday, February 2 2023","Thursday, February 2 2023",7:30 AM,12:05 PM,2023.02.07,608,Lufthansa,4 hr 35 min,None,None,None


In [7]:
import os
from azure.storage.blob import BlobServiceClient

connect_str = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

In [11]:
container_name = "mol"
container_client= blob_service_client.get_container_client(container_name)

In [14]:
from operator import contains


df.to_csv(output_csv_path)

blob_client = blob_service_client.get_blob_client(container=container_name, blob=output_csv_path)
print("\nUploading to Azure Storage as blob:\n\t" + output_csv_path)

with open(output_csv_path, "rb") as f:
    blob_client.upload_blob(f, overwrite=True)


Uploading to Azure Storage as blob:
	data\flights.csv
